# Lab 2: Linear Programming Forms
Implement conversion to normal and canonical forms using NumPy.

In [ ]:
import sys
import subprocess


import numpy as np

## LP Representation
Objective is max/min. Constraints use <=, >=, =. Variable signs: >=0, <=0, free.

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class LPProblem:
    c: np.ndarray
    A: np.ndarray
    b: np.ndarray
    sense: List[str]
    var_signs: List[str]
    objective: str
    var_names: List[str]

def apply_var_signs(c, A, var_signs, var_names=None):
    if var_names is None:
        var_names = [f'x{i+1}' for i in range(A.shape[1])]
    new_c = []
    new_cols = []
    new_names = []
    for i, sign in enumerate(var_signs):
        col = A[:, i]
        name = var_names[i]
        if sign == '>=0':
            new_c.append(c[i])
            new_cols.append(col)
            new_names.append(name)
        elif sign == '<=0':
            new_c.append(-c[i])
            new_cols.append(-col)
            new_names.append(f'{name}_neg')
        elif sign == 'free':
            new_c.append(c[i])
            new_c.append(-c[i])
            new_cols.append(col)
            new_cols.append(-col)
            new_names.append(f'{name}_plus')
            new_names.append(f'{name}_minus')
        else:
            raise ValueError(f'Unknown sign: {sign}')
    new_A = np.column_stack(new_cols) if new_cols else np.zeros((A.shape[0], 0))
    new_c = np.array(new_c, dtype=float)
    new_signs = ['>=0'] * len(new_c)
    return new_c, new_A, new_signs, new_names

def to_normal_form(lp: LPProblem) -> LPProblem:
    c = lp.c.astype(float).copy()
    A = lp.A.astype(float).copy()
    b = lp.b.astype(float).copy()
    if lp.objective == 'min':
        c = -c
    expanded = []
    for i, s in enumerate(lp.sense):
        row = A[i]
        rhs = b[i]
        if s == '=':
            expanded.append((row, rhs, '<='))
            expanded.append((row, rhs, '>='))
        else:
            expanded.append((row, rhs, s))
    rows = []
    rhs_list = []
    for row, rhs, s in expanded:
        if s == '>=':
            rows.append(-row)
            rhs_list.append(-rhs)
        else:
            rows.append(row)
            rhs_list.append(rhs)
    A2 = np.vstack(rows) if rows else np.zeros((0, A.shape[1]))
    b2 = np.array(rhs_list, dtype=float)
    c2, A3, signs, names = apply_var_signs(c, A2, lp.var_signs, lp.var_names)
    return LPProblem(c=c2, A=A3, b=b2, sense=['<='] * A3.shape[0], var_signs=signs, objective='max', var_names=names)

def to_canonical_form(lp: LPProblem) -> LPProblem:
    c = lp.c.astype(float).copy()
    A = lp.A.astype(float).copy()
    b = lp.b.astype(float).copy()
    if lp.objective == 'min':
        c = -c
    c2, A2, signs, names = apply_var_signs(c, A, lp.var_signs, lp.var_names)
    m = A2.shape[0]
    slack_cols = []
    slack_names = []
    for i, s in enumerate(lp.sense):
        if s == '<=':
            col = np.zeros(m)
            col[i] = 1.0
            slack_cols.append(col)
            slack_names.append(f's{i+1}')
        elif s == '>=':
            col = np.zeros(m)
            col[i] = -1.0
            slack_cols.append(col)
            slack_names.append(f's{i+1}')
        elif s == '=':
            continue
        else:
            raise ValueError(f'Unknown sense: {s}')
    if slack_cols:
        S = np.column_stack(slack_cols)
        A3 = np.column_stack([A2, S])
        c3 = np.concatenate([c2, np.zeros(S.shape[1])])
        names = names + slack_names
        signs = signs + ['>=0'] * S.shape[1]
    else:
        A3 = A2
        c3 = c2
    return LPProblem(c=c3, A=A3, b=b, sense=['='] * A3.shape[0], var_signs=signs, objective='max', var_names=names)

def summarize_lp(lp: LPProblem):
    print('objective:', lp.objective)
    print('c:', lp.c)
    print('A:')
    print(lp.A)
    print('b:', lp.b)
    print('sense:', lp.sense)
    print('var_signs:', lp.var_signs)
    print('var_names:', lp.var_names)

## Example from the task

In [ ]:
c = np.array([-1, 1, -2], dtype=float)
A = np.array(
    [
        [1, 0, 1],
        [1, 0, -1],
        [1, 1, 0]
    ],
    dtype=float,
)
b = np.array([1, 2, 10], dtype=float)
sense = ['<=', '>=', '=']
var_signs = ['>=0', '<=0', 'free']
var_names = ['x1', 'x2', 'x3']

lp = LPProblem(c=c, A=A, b=b, sense=sense, var_signs=var_signs, objective='min', var_names=var_names)

print('Original')
summarize_lp(lp)

print('
Normal form')
normal_lp = to_normal_form(lp)
summarize_lp(normal_lp)

print('
Canonical form')
canonical_lp = to_canonical_form(lp)
summarize_lp(canonical_lp)